In [1]:
import numpy as np
import pandas as pd
import xlwings as xw

import sqlalchemy
from sqlalchemy import create_engine
import os
import datetime as dt
import sqlite3
import unittest


pd.set_option('display.max_rows', 100)


 - Single occupancy management


 - 1f 18+ then police verification ref field
 

 - create a record history for new admission, updation of resident or electricity record
 
 add name in transaction history

 

 
 

 

 
 
 
 

In [2]:
temp = pd.read_excel("main.xlsx", sheet_name="DB", parse_dates=["DateofAdmission", "DateofBirth", "RentStartDate"], dtype="O")

NameError: name 'pd' is not defined

In [3]:
temp

NameError: name 'temp' is not defined

In [14]:
any(temp[["EnrollmentID", "Name", "ContactNumber"]].isna().any(axis=0))

True

In [56]:
temp["RoomNo"].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '101',
       '102', '103', '104', '105', '106', '107', '108', '109', '110',
       '111', '112', '113', '114', '115', '116', '117', '201', '202',
       '203', '204', '205', '206', '207', '208', '209', '210', '211',
       '212', '213', '214', '215', '216', '217'], dtype=object)

In [57]:
temp.columns.str.replace("Room_", "")

Index(['EnrollmentID', 'BedID', 'RoomNo', 'DateofAdmission', 'Name',
       'FathersName', 'DateofBirth', 'ContactNumber', 'OtherContact',
       'FathersContact', 'MothersContact', 'AdditionalContact', 'Email',
       'Address', 'BloodGroup', 'ResidentAadhar', 'FathersAadhar',
       'InstituteName', 'InstituteAddress', 'Course', 'Batch',
       'InstituteContact', 'InstituteContactOther', 'InstituteID',
       'GuardianName', 'GuardianRelation', 'GuardianContact1',
       'GuardianContact2', 'GuardianAddress', 'Rent', 'Deposit',
       'RentStartDate'],
      dtype='object')

In [1]:
temp[["DateofAdmission", "DateofBirth", "RentStartDate"]]

NameError: name 'temp' is not defined

In [ ]:
def _process_resident_exit(self, row: pd.Series, monthly_factor: float = 30):
        """Process the workflow when a resident exits"""

        curr_status = self.data_manager.load_current_status()
        if not curr_status[self.uid].isin([row[self.uid]]).any():
            raise ValueError(f"Inlavid Exit. {self.uid}:{row[self.uid]} does not exist in status")

        curr_bed = curr_status.loc[curr_status[self.uid] == row[self.uid], self.bed_id].squeeze()
        if curr_bed != row[self.bed_id]:
            raise ValueError(f"Invalid Exit. The bedID of exiting {self.uid} does not match this uid in status")


        row_with_calc = self._electricity_adjustment(adj_type="exit", row=row)
        row_with_calc = row_with_calc.rename(columns={"TransDate": "ExitDate","RoomElectricityReading": "ExitElectricityReading",})
        resident_info = self.data_manager.load_residents_info_table(filter_ids=row_with_calc[self.uid].values, filter_cols=[self.uid, "Name","Rent", "Deposit"])
        row_with_calc = row_with_calc.merge(resident_info, on=self.uid, how="left")

        row_with_calc["RentThruDate"] = row["RentThruDate"]
        row_with_calc["PrevDueAmount"] = row["PrevDueAmount"]
        row_with_calc["AdditionalCharges"] = row["AdditionalCharges"]
        row_with_calc["Comments"] = row["Comments"]

        row_with_calc["RentDays"] = (row_with_calc["RentThruDate"] - row_with_calc["LastRentCalcDate"]).dt.days
        row_with_calc["RentDue"] = (row_with_calc["RentDays"] * row_with_calc["Rent"] / monthly_factor)
        row_with_calc["ElectricityCharges"] = (row_with_calc["CumulativeElectConsumption"] * self.confs.elect_rate)

        row_with_calc["TotalAmountDue"] = row_with_calc["RentDue"] + row_with_calc["ElectricityCharges"] + row_with_calc["PrevDueAmount"] + row_with_calc["AdditionalCharges"]
        row_with_calc["NetAmountDue"] = (row_with_calc["TotalAmountDue"] - row_with_calc["Deposit"])
        self.data_manager.insert_final_settlement_record(row_with_calc)
        return row_with_calc


In [ ]:
# prev_adjustments = self.load_last_adjustments_table()

            # room_prev_adj = prev_adjustments.loc[prev_adjustments[self.roomID]==row[self.roomID]].copy()
            # room_prev_adj["TransDate"] = row['TransDate']
            # room_prev_adj["NewRoomElectricityReading"] = row["RoomElectricityReading"] 
            # room_prev_adj["RoomElectricityConsumption"] = room_prev_adj["NewRoomElectricityReading"] -  room_prev_adj["RoomElectricityReading"]
            # room_prev_adj["CumulativeElectConsumption"] = room_prev_adj["CumulativeElectConsumption"] + (room_prev_adj["RoomElectricityConsumption"] / uids_count_before_trans)
            
            # room_prev_adj =room_prev_adj.drop(["RoomElectricityReading","RoomElectricityConsumption"], axis=1).rename(columns={"NewRoomElectricityReading" : "RoomElectricityReading"})
            
            # # Saving the record for the exiting uid
            # exiting_record = room_prev_adj[room_prev_adj[self.uid]==row[self.uid]].copy()
            
            # # Removing the exiting uid from both 
            # room_prev_adj_post_trans = room_prev_adj[room_prev_adj[self.uid]!=row[self.uid]]
            
            # # Removing the room details from prev adjustments
            # updated_adjustments = prev_adjustments[prev_adjustments[self.roomID]!=row[self.roomID]].copy()
            # updated_adjustments = pd.concat([updated_adjustments, room_prev_adj_post_trans])
            # self.update_adjsutments_records(updated_adjustments, if_exists="replace")
            
            # # Changing the current status i.e removing the record of the exiting resident and updating current status
            # curr_status.loc[curr_status[self.uid].isin(uids_before_trans), "LastElectricityCalcDate"] = row["TansDate"]
            # curr_status.loc[curr_status[self.uid]==row[self.uid]] = np.nan
            # self.update_current_status(new_status=curr_status, if_exists="replace")